In [15]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, to_date, coalesce, lit, mode, when, count
import pandas as pd
from datetime import datetime, timedelta
from utils.session import get_cached_session, get_session
from datetime import date, timedelta

In [2]:
session = get_session()

In [19]:
zg_df.limit(10).to_pandas()

,ZONEGROUPCODE,ZONEGROUPNAME,ZONECODE,ZONENAME,STORECODE,PRIMARYIND
0,1,Base,1,SB AND ELKHART,2304,NaN
1,1,Base,1,SB AND ELKHART,2306,NaN
2,1,Base,1,SB AND ELKHART,2310,NaN
3,1,Base,1,SB AND ELKHART,2311,NaN
4,1,Base,1,SB AND ELKHART,2312,NaN
5,1,Base,1,SB AND ELKHART,2314,NaN
6,1,Base,1,SB AND ELKHART,2315,NaN
7,1,Base,1,SB AND ELKHART,2316,NaN
8,1,Base,1,SB AND ELKHART,2317,NaN
9,1,Base,1,SB AND ELKHART,2324,NaN


In [ ]:
ip_df = session.table('edl.phq.item_price')
irpd_df = session.table('edl.phq.item_retail_price_dt')
im_df = session.table('edl.phq.item_master')
pt_df = session.table('edl.phq.price_type')
zg_df = session.table('edl.phq.sn_rev_zonegrp')
item_df = session.table('edw.rtl.retail_item_vw')

#joins
df = irpd_df.join(
    ip_df,
    irpd_df['item_price_id'] == ip_df['item_price_id'],
    how = 'inner'
).join(
    im_df,
    ip_df['item_id'] == im_df['item_id'],
    how = 'inner'
).join(
    item_df,
    im_df['upc_ean'] == item_df['product_upc_13_cd'],
    how = 'inner'
).join(
    pt_df,
    irpd_df['pt_type'] == pt_df['pt_type'],
    how = 'inner'
).join(
    zg_df,
    irpd_df['store_id'] == zg_df['storecode']
)

#filters
df = df.filter(
    #(irpd_df['ip_start_date'] <= default_date) &
    #(irpd_df['ip_end_date'] >= default_date) &
    #(pt_df['pt_type'] == '1') &
    (ip_df['price_strategy'] != 0) &
    (zg_df['zonecode'] == 1) &
    (zg_df['zonegroupcode'] == 1)
).select(
    zg_df['zonename'],
    item_df['product_upc_13_cd'],
    irpd_df['v_id'],
    pt_df['description'],
    irpd_df['IP_UNIT_PRICE'],
    irpd_df['IP_PRICE_MULTIPLE'],
    irpd_df['IP_START_DATE'],
    irpd_df['IP_END_DATE']
)

#aggregations
result_df = df.group_by(
    col('zonename'),
    col('product_upc_13_cd'),
    col('v_id')
).agg(
    mode(when(col('description') == 'Regular', col('IP_UNIT_PRICE'))).alias('IP_UNIT_PRICE'),
    mode(when(col('description') == 'Regular', col('IP_PRICE_MULTIPLE'))).alias('IP_PRICE_MULTIPLE'),
    mode(when(col('description') == 'Regular', col('IP_START_DATE'))).alias('IP_START_DATE'),
    mode(when(col('description') == 'Regular', col('IP_END_DATE'))).alias('IP_END_DATE'),
    count(when(col('description') == 'Regular', 1)).alias('STORE_COUNT'),

    mode(when(col('description') != 'Regular', col('PT_DESCRIPTION'))).alias('PROMO_TYPE'),
    mode(when(col('description') != 'Regular', col('IP_UNIT_PRICE'))).alias('PROMO_UNIT_PRICE'),
    mode(when(col('description') != 'Regular', col('IP_PRICE_MULTIPLE'))).alias('PROMO_PRICE_MULTIPLE'),
    mode(when(col('description') != 'Regular', col('IP_START_DATE'))).alias('PROMO_START_DATE'),
    mode(when(col('description') != 'Regular', col('IP_END_DATE'))).alias('PROMO_END_DATE'),
    count(when(col('description') != 'Regular', 1)).alias('PROMO_STORE_COUNT')
)

In [37]:
df.show(10)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ZONENAME"  |"PRODUCT_UPC_13_CD"  |"l_0106_V_ID"  |"r_0113_DESCRIPTION"  |"l_0106_IP_UNIT_PRICE"  |"l_0106_IP_PRICE_MULTIPLE"  |"l_0106_IP_START_DATE"  |"l_0106_IP_END_DATE"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            |                     |               |                      |                        |                            |                        |                      |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [35]:
df

In [11]:
today = date.today()
days_since_saturday = (today.weekday() - 5) % 7
default_date = today - timedelta(days=days_since_saturday)
default_date

datetime.date(2025, 8, 23)

In [2]:
session = get_cached_session()

2025-08-27 17:22:35.542 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.048 
  command:

    streamlit run c:\Users\212739\OneDrive - SpartanNash\Documents\Data\Streamlit\Pricing App\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-27 17:22:36.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.557 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.563 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare 

In [20]:
df = session.table("SBX_BIZ.MARKETING.T_ITEM").select(
        col('"Anchor Group ID"'),
        col('"Item Description"'),
        col('"Product UPC"'),
        col('"Unit Size"'),
        col('"Group ID"'),
        col('"Category ID"'),
        col('"Brand"')
    ).to_pandas()

In [15]:
item_prices_df = session.table('EDL.PHQ.ITEM_PRICE').filter(
    (default_date >= to_date(col('IP_START_DATE'))) & 
        (default_date <= coalesce(to_date(col('IP_END_DATE')), lit('9999-12-31'))) &
    (col('PRICE_STRATEGY') == 11) & 
    (col('RECORD_STATUS') != 3)
)

item_prices_df.show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
item_prices_df = session.table('EDL.PHQ.ITEM_PRICE').filter(
    (col('PRICE_STRATEGY') == 11) & 
    (col('RECORD_STATUS') != 3)
)

item_prices_df.show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------